In [114]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
# import lightgbm
from lightgbm import LGBMRegressor
import datetime

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
rcParams['figure.figsize'] = (16, 8)
rc('font', family='AppleGothic')

In [115]:
train_path = './trainingdata'
infer_path = './inferencedata'
submission_path = './submission'
pil_path = './magok'

In [116]:
submission = pd.read_csv('./data/sample_submission.csv')
submission2 = pd.read_csv('./data/sample_submission.csv')

In [163]:
time_str = '20210101_232531'
train = pd.read_csv(
    os.path.join(train_path, f'training_{time_str}.csv')
)
X_test = pd.read_csv(
    os.path.join(infer_path, f'inference_{time_str}.csv')
)

In [164]:
train_day23 = pd.read_csv(
    os.path.join(train_path, f'training_{time_str}_day23.csv')
)
X_test_day23 = pd.read_csv(
    os.path.join(infer_path, f'inference_{time_str}_day23.csv')
)

# 필령 path

In [161]:
# 필령이형 path
train = pd.read_csv(
    os.path.join(pil_path, 'train_pilv2.csv')
)
X_test = pd.read_csv(
    os.path.join(pil_path, 'test_pilv2.csv')
)

In [102]:
train['Hour'] = train['Hour'].astype('category')
X_test['Hour'] = X_test['Hour'].astype('category')

In [103]:
X_train = train.drop(columns = ['1day_after_target', '2day_after_target'])
target_1 = train['1day_after_target']
target_2 = train['2day_after_target']

# Metric

In [119]:
def pinball(pred, actual):
    quantile_col = pred.columns.tolist()
    pred2 = pred.copy()
    actual2 = actual.copy()
    
    pred2.index = range(len(pred2))
    actual2.index = range(len(actual2))

    pinball_loss = []
    for item in quantile_col:
        pre_pinball = []
        for idx in range(len(pred2)):
            y = actual2.iloc[idx]
            z = pred2[item].iloc[idx]
            q = item
            
            if y >= z:
                value = (y - z)*q
                pre_pinball.append(value)
            else:
                value = (z - y)*(1 - q)
                pre_pinball.append(value)
        loss_mean = np.mean(pre_pinball)
        pinball_loss.append(loss_mean)
    
    fin = np.mean(pinball_loss)
#     print(f'Pinball Loss: {fin}')
    
    return pinball_loss         

# Inference

In [165]:
X_train = train.drop(columns = ['1day_after_target', '2day_after_target'])
target_1 = train['1day_after_target']
target_2 = train['2day_after_target']

In [166]:
X_train_day23 = train_day23.drop(columns = ['1day_after_target', '2day_after_target'])
target_1_day23 = train_day23['1day_after_target']
target_2_day23 = train_day23['2day_after_target']

In [121]:
def LGBM_reg(X_train,y_train, test):
    quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    # 랜덤모수 샘플 추론이 왜 안되지;;;
    # fold = StratifiedKFold(n_splits=5, shuffle = True)
    
    fold = KFold(n_splits=7, shuffle = True)

    oof_preds = np.zeros([X_train.shape[0], len(quantiles)])
    
    feature_importance_df = pd.DataFrame()
    
    test_df = pd.DataFrame(np.zeros([test.shape[0], len(quantiles)]))
    test_df.columns = quantiles
    
    fold_metric = []
    # training
    target = y_train
    for fold_, (train_idx, valid_idx) in enumerate(
        fold.split(np.array(X_train), target)
    ):

        train_x, train_y = X_train.iloc[train_idx], target.iloc[train_idx]
        valid_x, valid_y = X_train.iloc[valid_idx], target.iloc[valid_idx]

        oof_df = pd.DataFrame()
        for idx, q in enumerate(quantiles):
            print(f'\nquantile: {q}\n')
            model = LGBMRegressor(objective='quantile',
                                  alpha=q,
                                  n_estimators=10000, 
                                  bagging_fraction=0.7, 
                                  learning_rate=0.027, 
                                  subsample=0.7
                                 )   

            model.fit(train_x, train_y, eval_metric = ['quantile'], 
              eval_set=[(valid_x, valid_y)], early_stopping_rounds=300, verbose=1500)

            oof_preds[[valid_idx],idx] = \
                model.predict(valid_x).round(2)

            oof_df[q] = model.predict(valid_x).round(2)
            test_df[q] += model.predict(test).round(2) / fold.n_splits
            
        metric = pinball(oof_df, valid_y)
        loss = np.mean(metric)
        print(f'\n FOLD {fold_}의 total pinball loss: {loss}\n')
        fold_metric.append(metric)
        # feature importance
        
        print('\n===================================================')
        print(f'FOLD {fold_} Success')
        print('===================================================\n')
        
    return oof_preds, fold_metric, test_df

In [122]:
oof_preds, fold_metric, test_df = \
    LGBM_reg(X_train, target_1, X_test)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[957]	valid_0's quantile: 1.33845

quantile: 0.2

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[981]	valid_0's quantile: 2.09358

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.4818
Early stopping, best iteration is:
[2272]	valid_0's quantile: 2.46489

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.61179
Early stopping, best iteration is:
[2123]	valid_0's quantile: 2.59667

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.56342
[3000]	valid_0's quantile: 2.52845
[4500]	valid_0's quantile: 2.51446
[6000]	valid_0's quantile: 2.50401
Early stopping, best iteration is:
[7072]	valid_0's quantile: 2.50086

quantile: 0.6

Training until validation scores don'

Early stopping, best iteration is:
[5443]	valid_0's quantile: 1.88067

quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.38244
[3000]	valid_0's quantile: 1.375
Early stopping, best iteration is:
[3349]	valid_0's quantile: 1.37446

quantile: 0.9

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 0.756867
Early stopping, best iteration is:
[1959]	valid_0's quantile: 0.756114

 FOLD 3의 total pinball loss: 1.9090522483349477


FOLD 3 Success


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[527]	valid_0's quantile: 1.36235

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.13639
[3000]	valid_0's quantile: 2.12538
[4500]	valid_0's quantile: 2.11709
[6000]	valid_0's quantile: 2.11358
[7500]	valid_0's quantile: 2.11126
Early stopping, best iteration is:
[7736]	valid_0's quantile: 

In [123]:
oof_preds2, fold_metric2, test_df2 = \
    LGBM_reg(X_train, target_2, X_test)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[373]	valid_0's quantile: 1.40493

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.24313
Early stopping, best iteration is:
[1884]	valid_0's quantile: 2.23567

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.6224
[3000]	valid_0's quantile: 2.59974
[4500]	valid_0's quantile: 2.5919
Early stopping, best iteration is:
[4218]	valid_0's quantile: 2.59159

quantile: 0.4

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[1179]	valid_0's quantile: 2.73386

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.65295
[3000]	valid_0's quantile: 2.62022
[4500]	valid_0's quantile: 2.60374
Early stopping, best iteration is:
[4766]	valid_0's quantile: 2.60192

quantile: 0.6

Tr

[4500]	valid_0's quantile: 2.30111
[6000]	valid_0's quantile: 2.29396
[7500]	valid_0's quantile: 2.28803
[9000]	valid_0's quantile: 2.28532
Did not meet early stopping. Best iteration is:
[9972]	valid_0's quantile: 2.28342

quantile: 0.7

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.92572
[3000]	valid_0's quantile: 1.91038
Early stopping, best iteration is:
[4122]	valid_0's quantile: 1.90477

quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.38298
[3000]	valid_0's quantile: 1.37794
Early stopping, best iteration is:
[3718]	valid_0's quantile: 1.37599

quantile: 0.9

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 0.772539
Early stopping, best iteration is:
[2255]	valid_0's quantile: 0.771234

 FOLD 3의 total pinball loss: 1.947112472393584


FOLD 3 Success


quantile: 0.1

Training until validation scores don't improve for 300 rounds
[1500]	valid

In [132]:
oof_preds_day23, fold_metric_day23, test_df_day23 = \
    LGBM_reg(X_train_day23, target_1_day23, X_test_day23)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[320]	valid_0's quantile: 1.33585

quantile: 0.2

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[630]	valid_0's quantile: 2.11922

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.4413
Early stopping, best iteration is:
[2427]	valid_0's quantile: 2.42883

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.59625
Early stopping, best iteration is:
[1450]	valid_0's quantile: 2.59506

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.50481
Early stopping, best iteration is:
[1474]	valid_0's quantile: 2.50436

quantile: 0.6

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.26622
Early stopping, best iteration is:
[2214]	val

Early stopping, best iteration is:
[751]	valid_0's quantile: 1.41048

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.26393
[3000]	valid_0's quantile: 2.25094
Early stopping, best iteration is:
[2982]	valid_0's quantile: 2.2505

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.68981
[3000]	valid_0's quantile: 2.66285
Early stopping, best iteration is:
[4040]	valid_0's quantile: 2.65274

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.81653
[3000]	valid_0's quantile: 2.77512
[4500]	valid_0's quantile: 2.76
[6000]	valid_0's quantile: 2.75664
Early stopping, best iteration is:
[5773]	valid_0's quantile: 2.75634

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.72551
[3000]	valid_0's quantile: 2.6937
[4500]	valid_0's quantile: 2.68415
[6000]	valid_0's quantile: 2.

In [133]:
oof_preds2_day23, fold_metric2_day23, test_df2_day23 = \
    LGBM_reg(X_train_day23, target_2_day23, X_test_day23)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[306]	valid_0's quantile: 1.3839

quantile: 0.2

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[801]	valid_0's quantile: 2.23525

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.68552
[3000]	valid_0's quantile: 2.66403
Early stopping, best iteration is:
[3674]	valid_0's quantile: 2.66403

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.80587
[3000]	valid_0's quantile: 2.79089
Early stopping, best iteration is:
[3447]	valid_0's quantile: 2.79089

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.6992
Early stopping, best iteration is:
[2596]	valid_0's quantile: 2.6668

quantile: 0.6

Training until validation scores don't improve for 300 rounds
[1500]	valid

Early stopping, best iteration is:
[1653]	valid_0's quantile: 2.53535

quantile: 0.6

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.26473
[3000]	valid_0's quantile: 2.24023
Early stopping, best iteration is:
[4037]	valid_0's quantile: 2.23109

quantile: 0.7

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.87959
Early stopping, best iteration is:
[2213]	valid_0's quantile: 1.871

quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.38196
[3000]	valid_0's quantile: 1.37461
Early stopping, best iteration is:
[3273]	valid_0's quantile: 1.37388

quantile: 0.9

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 0.767154
Early stopping, best iteration is:
[2029]	valid_0's quantile: 0.765629

 FOLD 4의 total pinball loss: 1.9360789402691871


FOLD 4 Success


quantile: 0.1

Training until validation scores don't improv

# submission

In [124]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = test_df.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = test_df2.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [136]:
submission2.loc[submission2.id.str.contains("Day7"), "q_0.1":] = test_df_day23.sort_index().values
submission2.loc[submission2.id.str.contains("Day8"), "q_0.1":] = test_df2_day23.sort_index().values
submission2

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Ensemble

In [159]:
submission3 = submission[['id']]
sub_value = submission.iloc[:,1:] * 0.7 + submission2.iloc[:,1:] * 0.3

submission3 = pd.concat([submission3,sub_value],axis = 1)

# 제출

In [157]:
# time_str = datetime.datetime.strftime(
#     datetime.datetime.today(),
#     '%Y%m%d_%H%M%S'
# )

# training 데이터 저장한 시간이랑 맞춰서 사용
submission.to_csv(
    os.path.join(submission_path, f'제출용_{time_str}.csv'), 
    index=False
)

In [160]:
# 앙상블 결과 제출
submission3.to_csv(
    os.path.join(submission_path, f'제출용_{time_str}_ensemble_가중평균.csv'), 
    index=False
)